In [ ]:
project_path = "drive/My Drive/VGSI" # use the shared folder https://drive.google.com/drive/u/1/folders/1hjjcNSUSqv8AbA7R-5lIKmui-ySCEWJw
import os
import pickle
from tqdm import tqdm
import numpy as np
from keras.layers import Dense, Concatenate, Input, Dropout, Bidirectional, LSTM, Dropout
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dropout
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME='VGSI'
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/

NotImplementedError: ignored

In [ ]:
# Samples
sample_multi = pickle.load(open("samples/wikihow/random_sampling/train_goal_random.p", "rb"))
test_sample_multi = pickle.load(open("samples/wikihow/random_sampling/test_goal_random.p", "rb"))
print(len(test_sample_multi))
test_sample_multi=test_sample_multi[:8000]
print(len(test_sample_multi))
train_sample_multi = sample_multi[:-1000]
val_sample_multi = sample_multi[-1000:]

FileNotFoundError: ignored

In [ ]:
# Features
encoded_goals = pickle.load(open("features/wikihow/encoded_goals.p", "rb" ))
print("goal loaded!")
encoded_images_1 = pickle.load(open( "features/wikihow/encoded_images_1.p", "rb" ))
print("image loaded!")
encoded_images_2 = pickle.load(open("features/wikihow/encoded_images_2.p", "rb" ))
print("image2 loaded!")
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res
encoded_images = Merge(encoded_images_1, encoded_images_2)

FileNotFoundError: ignored

# Model training

In [ ]:
def build_text_network(input_shape, embedding_size):
    network = Sequential()
    network.add(Dense(1024, input_shape=input_shape, activation='relu',
                   kernel_regularizer=l2(1e-1),
                   kernel_initializer='he_uniform'))
    network.add(Dropout(0.5))
    network.add(Dense(embedding_size, activation=None,
                   kernel_regularizer=l2(1e-2),
                   kernel_initializer='he_uniform'))

    #Force the encoding to live on the d-dimentional hypershpere
    network.add(Lambda(lambda x: K.l2_normalize(x,axis=-1)))

    return network

def build_img_network(input_shape, embedding_size):
    network = Sequential()
    network.add(Dense(1024, input_shape=input_shape, activation='relu',
                   kernel_regularizer=l2(1e-1),
                   kernel_initializer='he_uniform'))
    network.add(Dropout(0.5))
    network.add(Dense(embedding_size, activation=None,
                   kernel_regularizer=l2(1e-2),
                   kernel_initializer='he_uniform'))

    #Force the encoding to live on the d-dimentional hypershpere
    network.add(Lambda(lambda x: K.l2_normalize(x,axis=-1)))

    return network

@keras.saving.register_keras_serializable()
class TripletLossLayer(Layer):
    def __init__(self, **kwargs):
        super(TripletLossLayer, self).__init__(**kwargs)

    def triplet_loss(self, inputs):
        flag=False
        anchor, positive, negative = inputs
        p_dist = K.sum(K.square(anchor-positive), axis=-1)
        n_dist = K.sum(K.square(anchor-negative), axis=-1)
        #print(p_dist.numpy(),n_dist.numpy())
        if p_dist < n_dist - 0.35:
            flag=True
        return flag,K.sum(K.maximum(p_dist - n_dist + 0.35, 0), axis=0)

    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

text_network = build_text_network((768,), 1024)
img_network = build_img_network((2048,), 1024)
anchor_input = Input(768, name="anchor_input")
positive_input = Input(2048, name="positive_input")
negative_input = Input(2048, name="negative_input")
encoded_a = text_network(anchor_input)
encoded_p = img_network(positive_input)
encoded_n = img_network(negative_input)
loss_layer = TripletLossLayer(name='triplet_loss_layer')([encoded_a,encoded_p,encoded_n])
network_train = Model(inputs=[anchor_input,positive_input,negative_input],outputs=loss_layer)
optimizer = Adam(lr = 0.0001)
network_train.compile(loss=None,optimizer=optimizer)
network_train.summary()

In [ ]:
train_sample_tri = []
for sample in tqdm(train_sample_multi):
  a = sample[0]
  p = sample[1][sample[-1]]
  for i in range(4):
    if i != sample[-1]:
      n = sample[1][i]
      train_sample_tri.append([a, p, n])

val_sample_tri = []
for sample in tqdm(val_sample_multi):
  a = sample[0]
  p = sample[1][sample[-1]]
  for i in range(4):
    if i != sample[-1]:
      n = sample[1][i]
      val_sample_tri.append([a, p, n])

X_val_a = []
X_val_p = []
X_val_n = []
y_val = []
for sample in val_sample_tri:
  X_val_a.append(encoded_goals[sample[0]].squeeze())
  X_val_p.append(encoded_images[sample[1]].squeeze())
  X_val_n.append(encoded_images[sample[2]].squeeze())
  y_val.append(1)
X_val_a = np.array(X_val_a)
X_val_p = np.array(X_val_p)
X_val_n = np.array(X_val_n)
y_val = np.array(y_val)

In [ ]:
def data_loader(train_sample, batch_size):
  count = 0
  X_train_a_batch = []
  X_train_p_batch = []
  X_train_n_batch = []
  y_train_batch = []
  while True:
    for sample in train_sample:
      X_train_a_batch.append(encoded_goals[sample[0]].squeeze())
      X_train_p_batch.append(encoded_images[sample[1]].squeeze())
      X_train_n_batch.append(encoded_images[sample[2]].squeeze())
      y_train_batch.append(1)
      count += 1
      if count > batch_size:
        X_train_a_batch = np.array(X_train_a_batch)
        X_train_p_batch = np.array(X_train_p_batch)
        X_train_n_batch = np.array(X_train_n_batch)
        y_train_batch = np.array(y_train_batch)
        yield [X_train_a_batch, X_train_p_batch, X_train_n_batch], y_train_batch
        count = 0
        X_train_a_batch = []
        X_train_p_batch = []
        X_train_n_batch = []
        y_train_batch = []

In [ ]:
history = network_train.fit(data_loader(train_sample_tri, 10000), validation_data=([X_val_a, X_val_p, X_val_n], y_val), steps_per_epoch=len(train_sample_tri)/10000, epochs=20, shuffle=True)

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
network_train.save("/content/drive/MyDrive/triplet_model.h5")

# Model load & evaluation


In [ ]:
loaded_model = load_model("/content/drive/MyDrive/triplet_model.h5", custom_objects={'TripletLossLayer': TripletLossLayer})

In [ ]:
test_sample_tri = []
for sample in tqdm(test_sample_multi):
  a = sample[0]
  p = sample[1][sample[-1]]
  for i in range(4):
    if i != sample[-1]:
      n = sample[1][i]
      test_sample_tri.append([a, p, n])

X_test_a_batch,X_test_p_batch,X_test_n_batch=[],[],[]
for sample in test_sample_tri:
      X_test_a_batch.append(encoded_goals[sample[0]].squeeze())
      X_test_p_batch.append(encoded_images[sample[1]].squeeze())
      X_test_n_batch.append(encoded_images[sample[2]].squeeze())

In [ ]:
correct_preds=0
for text_anchor, image_positive, image_negative in zip(X_test_a_batch,X_test_p_batch,X_test_n_batch):
  flag,dist=network_train.predict([np.array(text_anchor).reshape(1,-1), np.array(image_positive).reshape(1,-1), np.array(image_negative).reshape(1,-1)])
  if flag==True:
    correct_preds=correct_preds+1
    print(correct_preds)

In [ ]:
total_samples=len(test_sample_tri)

In [ ]:
accuracy = correct_preds / total_samples

In [ ]:
p1=loaded_model.predict([np.array(encoded_test_anchor).reshape(1,-1), np.array(image_positive).reshape(1,-1), np.array(encoded_test_n).reshape(1,-1)])

In [ ]:
test_loss = network_train.evaluate([X_test_a_batch, X_test_p_batch, X_test_n_batch])
print(f'Test Loss: {test_loss}')

In [ ]:
X_test_a_batch